# Tomography as a Constrained Quadratic Problem on a Quantum Annealer

To run an optimization of a constrained quadratic model on DWaves Ocean platform, we have to initialize a quadratic model, optionally add constraints and then transfer the problem setting to the dedicated hybrid solver. We utilize numpy for classical preproccesing computations and solution calculation. To simulate a tomographic problem we utilize skimage to implement a Radon transform. To formulate our problem for the DWave system, we initialize a dimod ConstrainedQuadraticModel with dimod Integer variables. Subsequently, we use the LeapHybridCQMSampler to sample the solution on the DWave quantum annealer. As Ocean does not yet support matrix operations, we import and use Sympy [3] to formulate our problem description.

In [ ]:
# Copyright © Siemens Medical Solutions USA, Inc., 2021. All rights reserved. Unauthorized reproduction prohibited.

# Classical numerical library
import numpy as np
# Symbolic matrix computations
from sympy import sympify, Matrix
# Tomographic problem 
from skimage.transform import radon, iradon, resize
# Radon transform as matrix operation
from tomography_radon import radon_matrix, get_reconstruction_circle
# Shepp-Logan phantom
from skimage.data import shepp_logan_phantom
# Constrained quadratic model
from dimod import ConstrainedQuadraticModel, Integer, Real
# Hybrid quantum annealing sampler
from dwave.system import LeapHybridCQMSampler

## Introduction to Tomography

Tomographic reconstruction is a multidimensional inverse problem, where the problem is to estimate an object by only having projection views of the object. In Computed Tomography the projection views are captured by placing an x-ray source on one part of the object and a detector on the other side. The x-rays are attenuated by the matter and captured by the detector. In Single Photon Emission Computed Tomography (SPECT) the situation changes as the radiation source is in the patient, therefore collimators are used in front of the detectors to capture parallel gamma rays. One projection view can be thought of as an integration/summation of the rotated object. The simple projection process and corresponding matrix notation is showcased below:

<p align="center">
 <img src="Illustrations/QuantumAnnealingTomography.PNG">
</p>

Here $Y$ represents our acquired projection values and $X$ is our object to be imaged. We use a zero-based number indexing for vectors and matrices to comply with Python defaults. We can write both $X$ and $Y$ in vector representation. The relationship between our object image $X$ and projections $Y$ can be described by the system matrix $M$. The system matrix $M$ is defined by the imaging geometry and describes each entry the conditional probability that an emission of $X_i$ was detected in $Y_j$.

$ Y = M * X =\begin{pmatrix} Y_0 \\ Y_1 \\ Y_2 \\ Y_3 \\ Y_4 \\ Y_5 \\ Y_6 \\ Y_7 \\ Y_8  \end{pmatrix} = M *  \begin{pmatrix} X_0 & X_1 & X_2 & X_3 & X_4 & X_5 & X_6 & X_7 & X_8  \end{pmatrix}^T  \; \; \; \; \; \text{ with } \; \; \; \; \; M =  p(detected \: in \: y  \: | emitted \: in \: x) = \begin{pmatrix} 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0  \\
                 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0  \\
                 1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 & 1  \\
                 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0  \\
                 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0  \\
                 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1  \\
 \end{pmatrix}
$

 The projection data can be obtained by the well known Radon transform. In the process of tomographic imaging, multiple views from different angles are captured to later reconstruct the object. The inverse problem rises in complexity when fewer views are chosen. In principal, the solution to the inverse tomographic problem is a matrix inversion of $M$ to solve for:
 
  $X = M^{-1}Y$ 

However, M in reality is sparse, very large, ill-posed, non-square and/or singular. Moreover, the projection values suffer from various source of noise e.g. Photon noise. Therefore, analytical methods like Filtered Backprojection (FBP) or iterative methods like Maximum Likelihood Expectation Maximization or Conjugate Gradient are used to reconstruct the original volume. Below, a realistic example of the imaged object and the corresponding projections is shown acquired with the Radon transform. 

<p align="center">
 <img src="Illustrations/Tomography.PNG">
</p>

## Create tomographic toy problem
1. Create the object to be imaged.

In [ ]:
image_size = 16
image_intensity_max = 16
image = shepp_logan_phantom()
# Resize and binarize
image = resize(image, (image_size, image_size), anti_aliasing=True)
image *= image_intensity_max
image = image.astype(int)
mask = get_reconstruction_circle(image.shape).astype(int)
image = image * mask

2. Create the noise-free system matrix

In [ ]:
system = radon_matrix(image.shape[0], image.shape[1])
print(np.linalg.cond(system))

2. Create the noise-free sinogram at equally spaced angles

In [ ]:
sinogram = system @ image.flatten()

4. Plot simulated object and sinogram

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))
ax1.set_title("Original")
ax1.imshow(image, cmap=plt.cm.Greys_r)

plt_sinogram = sinogram.reshape(image.shape, order='F')
print(plt_sinogram.shape)
dx, dy = 0.5 * 180.0 / max(image.shape), 0.5 / plt_sinogram.shape[0]

ax2.set_title("Radon transform\n(Sinogram)")
ax2.set_xlabel("Projection angle (deg)")
ax2.set_ylabel("Projection position (pixels)")

ax2.imshow(plt_sinogram, cmap=plt.cm.Greys_r,
           extent=(-dx, 180.0 + dx, -dy, plt_sinogram.shape[0] + dy),
           aspect='auto')
fig.tight_layout()
fig.patch.set_facecolor('white')
plt.show()

## Matrix Inversion as a QUBO problem
This notebook is an extension and update to the implementation of the paper "Floating-Point Calculations on a Quantum Annealer: Division and Matrix Inversion" by M. Rogers and R. Singleton, 2020 [1]. 

The paper describes an approach to floating-point calculations on a quantum annealer. Specifically, they introduce a binary representation of the floating point variables. This representation is discrete, as they demonstrate their technique for 4 bit and 8 bit accuracy. With this measure they explain and derive the inversion of a matrix as a Quadratic Unconstrained Binary Optimization problem (QUBO), which is suitable to run on a DWave quantum annealer. 

We plan to expand the discrete (limited floating-point accuracy) to an integer problem solvable on the new hybrid solvers from DWave. In addition to solving integer style problems, the new hybrid solvers allow the integration of mathematical constraints. In the context of tomographic reconstruction for example non-negativity constraints are frequently used.

### Formulating the problem for the quantum annealer

The main challenge in formulating problems for quantum annealers is the description as a quadratic binary problem. Previously Rogers and Singleton have described matrix inversion as a quadratic binary problem for floating-point precision [1].   

Scalar notation of a QUBO problem: 

\begin{equation} E_{qubo}(a_i, b_{i,j}; q_i) = \sum_{i} a_i q_i + \sum_{i < j} b_{i, j} q_i q_j 
\end{equation} 


- $q_i$: Unknown binary variable (0, 1) we want to optimize for 
- $a_i$: Qubit bias/weight (real number)
- $b_{i,j}$: Coupling strength between spins $i$ and $j$ (real number)




Matrix (NxN) inversion is defined as: 

\begin{equation} M \cdot x = y \rightarrow x=M^{-1} \cdot y 
\end{equation} 

Formulate matrix inversion as a quadratic minimization problem with its minimum being the matrix inverse:

\begin{equation} H(x) = (Mx-y)^2 = \sum_{ijk=1}^{N} M_{ki} M_{kj} x_{i} x_{j} - 2 \sum_{ij=1}^{N} y_j M_{ji} x_{i} + \| y \|^2 = x^T M^T M x - x^T M^T y - y^t M x + y^T y
\end{equation} 

In matrix inversion and solving a linear system we usually solve for a vector of numbers which are typically not binary. Previously the common practice to obtain a floating point or integer representation of each component of x was to expand in powers of 2 multiplied by the boolean-valued variables $q_r^i \in \{0,1\}$:

\begin{equation} \chi^i = \sum_{r=0}^{R-1} 2^{-r} q_{r}^{i}
\end{equation} 

\begin{equation} x^i = 2\chi^i -1
\end{equation}

From the equation above we can see that the precision of the number heavily depends on the size or R (the number of bits used to represent the number). In the paper by Rogers and Singleton they opt for an R or 4, representing numbers as multiplies of 0.25 in the range of -1 to 3, this leads to a total number of 16 discrete values. 

The new constrained quadratic models (CQM) [2] running on DWaves Hybrid Samplers [3] enable the use of integer values in the quadratic model and therefore drastically expand solution possibilities. Further the possibility of introducing constraints is given. The new constrained quadratic model can be written as:

\begin{equation} H[q] = \sum_{i=1}^{N} a_i x_i + \sum_{i=1}^{N} \sum_{i \neq j}^{N}   b_{ij} x_i x_j  + c
\end{equation} 

- $x_i$: Unknown integer/binary variable we want to optimize for 
- $a_i$: Linear weight (real number)
- $b_{i,j}$: Quadratic term between $i$ and $j$ (real number)
- $c$: Inequality/equality constraints


## Implementation of the tomographic problem for the quantum annealer

### Problem formulation with sympy
We track our solution variables $x_i$ with the vector components. 

In [ ]:
M = Matrix(system.tolist())
x_list = []
for i in range(image.shape[0]*image.shape[1]):
    x_list.append(sympify(str('x'+str(i))))
X = Matrix(x_list)
Y = Matrix(sinogram.tolist())
M_C = system
Y_C = sinogram

Reformulate the objective function into components of x as seen above: 


$H(x) = ||Mx-y||^2$

In [ ]:
A = M.multiply(X)-Y
result = str(A.T.multiply(A)[0, 0])

In [ ]:
non_zero_entries = []
for i in reversed(range(len(X))):
    tmp_res = result
    result = result.replace('x'+str(i), 'x['+str(i)+']')
    if result != tmp_res:
        non_zero_entries.append(i)

Paste the above printed reformulation into cqm.set_objective(*printed reformulation*)

In [ ]:
#Constraints
lowerBound = 0
upperBound = 16

#### Create the constrained quadratic model

In [ ]:
# Create the quadratic model
cqm = ConstrainedQuadraticModel()
# Create a list of integer variables for each solution component
x = []

# Add constraints, here non-negativity and range constraint
for c in range(len(X)):
    if c in non_zero_entries:
        x.append(Integer('x'+str(c), lower_bound=lowerBound, upper_bound=upperBound))
    else: 
        x.append(Integer('x'+str(c), lower_bound=0, upper_bound=0))

# Reformulated objective function to minimize here
program = 'cqm.set_objective(' + result + ')'
exec(program)

# Substitute integers with self loops
cqm.substitute_self_loops()

#### Run the constrained quadratic model on the hybrid sampler

In [ ]:
# Sample from the Hybrid Solver on our constrained quadratic model
sampleset = LeapHybridCQMSampler().sample_cqm(cqm, label="Tomo Inversion Problem")
# Filter feasible solutions
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible) 

In [ ]:
import json
with open('sampleset_test_shepplogan_' + str(image_size) + 'x' + str(image_size) + '.json', 'w') as json_file:
    json.dump(feasible_sampleset.to_serializable(), json_file)

In [ ]:
print(feasible_sampleset)
quantum_reconstructions = np.zeros(( len(feasible_sampleset), system.shape[1],))
index = np.arange(0, system.shape[1])
for s, r in zip(feasible_sampleset.data(fields=['sample', 'energy'], sorted_by='energy'), range(len(feasible_sampleset))):
    for i in non_zero_entries:
        search = 'x' + str(i)
        quantum_reconstructions[r, i] = s.sample[search]


### Classical solution
Solution using the classical inverse approximation.

In [ ]:
print("Classical solution: ", np.linalg.pinv(M_C).dot(Y_C))
print("Condition number: ", np.linalg.cond(M_C))

Solution using filtered backprojection.

In [ ]:
reconstruction_fbp = iradon(plt_sinogram, theta=np.linspace(0., 180., image.shape[0], endpoint=False), filter_name='ramp')
error = reconstruction_fbp - image
print(f'FBP rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}')
imkwargs = dict(vmin=-0.2, vmax=0.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)
ax1.set_title("Reconstruction\nFiltered back projection")
shw0 = ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.set_title("Reconstruction error\nFiltered back projection")
shw1 = ax2.imshow(reconstruction_fbp - image, cmap=plt.cm.Greys_r, **imkwargs)
fig.patch.set_facecolor('white')
# add space for colour bar
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.15, 0.04, 0.7])
fig.colorbar(shw1, cax=cbar_ax, fraction=0.0046)
plt.show()

Solution using quantum annealing.

In [ ]:
reconstruction_fbp = quantum_reconstructions[0].reshape(image.shape)
error = reconstruction_fbp - image
print(f'QAR rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}')
imkwargs = dict(vmin=-0.2, vmax=0.2)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5), sharex=True, sharey=True)
ax1.set_title("Reconstruction\nQuantum Annealing")
shw0 = ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
ax2.set_title("Reconstruction error\nQuantum Annealing")
shw1 = ax2.imshow(reconstruction_fbp - image, cmap=plt.cm.Greys_r, **imkwargs)
fig.patch.set_facecolor('white')
# add space for colour bar
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.15, 0.04, 0.7])
fig.colorbar(shw1, cax=cbar_ax, fraction=0.0046)
plt.show()

## References 
- [1] Rogers, Michael L., and Robert L. Singleton Jr. "Floating-point calculations on a quantum annealer: Division and matrix inversion." Frontiers in Physics 8 (2020): 265.
- [2] https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/constrained.html
- [3] https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html
- [4] https://docs.sympy.org/latest/index.html